# The Qmod Workshop - Part 3: Execution Flows

This is the third part of the Qmod workshop, covering exercises 11 and 12. Make sure to go through Part 1 and 2 before continuing with this notebook.

### Exercise 11 - Execution with Parameters

In this exercise, we will modify the manually created state preparation function from the previous exercise to accept a rotation angle as a parameter.

1. Start by modifying the signature of the main function to be as follows:

In [ ]:
@qfunc
def main(rotation_angle: QParam[float], res: Output[QArray[QBit]]) -> None:
    pass

2. Pass the rotation angle as a parameter to the controlled RY instead of using `pi/3` directly.
3. Define the following quantum constant, which will serve as the list of execution parameters:

In [ ]:
angle_vals = QConstant("angle_vals", List[float], [pi / 3, pi / 2])

4. Create a cmain function that for each rotation angle, it calls the sample function and saves the result.
5. Execute the circuit and make sure the results are as expected (statistics from two runs should appear. What happens for `pi/2`?).
6. **Bonus**: try to add other values to the list and observe the results.

In [ ]:
from classiq import *

# Your code here:


qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

### Exercise 12 - VQE

The Variational Quantum Eigensolver is an algorithm that finds the minimal eigenvalue of a matrix by executing a parametric circuit (also referred to as an ansatz), estimating the expected value of the matrix for the state the circuit creates (from the distribution received by the execution), and using a classical optimizer to select the next set of parameters for the circuit, until reaching convergence (or exceeding a set amount of maximum iterations).

The estimation of the expectation value is done on Pauli based matrices, so any matrix we want to perform this operation on, need to be decomposed into a sum of Pauli terms.

In this exercise, we will create a simple VQE algorithm that estimates the minimal eigenvalue of a 2x2 matrix.
Fill in the gaps in the following snippet to find the minimal eigenvalue and it corresponding eigenstate for

`[[1, -1], [-1, 0]] = 1/2*I + 1/2*Z - X`


In [ ]:
from classiq import *

HAMILTONIAN = QConstant(...)


@qfunc
def main(q: Output[QBit], angles: QParam[Array[float, 3]]) -> None:
    allocate(1, q)
    U(angles[0], angles[1], angles[2], 0, q)


@cfunc
def cmain() -> None:
    res = vqe(
        hamiltonian=...,
        maximize=...,
        initial_point=[],
        optimizer=Optimizer.COBYLA,
        max_iteration=1000,
        tolerance=0.001,
        step_size=0,
        skip_compute_variance=False,
        alpha_cvar=1.0,
    )
    save({"result": res})


qmod = create_model(main, classical_execution_function=cmain)
qprog = synthesize(qmod)
show(qprog)
res = execute(qprog)
vqe_result = res.result()[0].value
print(vqe_result.energy, vqe_result.optimal_parameters, vqe_result.eigenstate)

Note:
The U gate is a general rotation matrix on a single qubit, so the given model creates an ansatz that spans all of the space for a single qubit, and thus gives us a full search space for this specific problem.

## Solutions

### Exercise 11

In [ ]:
# Solution to exercise 11:


from typing import List

from classiq import *
from classiq.qmod.symbolic import pi


@qfunc
def main(rotation_angle: QParam[float], res: Output[QArray[QBit]]) -> None:
    x: QArray[QBit] = QArray("x")
    allocate(3, x)
    hadamard_transform(x)

    ls_bit = QBit("ls_bit")
    ms_bits = QNum("ms_bits", 2, False, 0)
    bind(x, [ls_bit, ms_bits])

    quantum_if(ms_bits == 1, lambda: RY(rotation_angle, ls_bit))

    bind([ls_bit, ms_bits], res)


angle_vals = QConstant("angle_vals", List[float], [pi / 3, pi / 2])


@cfunc
def cmain() -> None:
    result = sample({"rotation_angle": angle_vals[0]})
    save({"result": result})
    result = sample({"rotation_angle": angle_vals[1]})
    save({"result": result})


model = create_model(main, classical_execution_function=cmain)
qprog = synthesize(model)
show(qprog)

### Exercise 12

In [ ]:
# Solution to exercise 12:


from typing import List

from classiq import *

HAMILTONIAN = QConstant("HAMILTONIAN", List[PauliTerm], [...])


@qfunc
def main(q: Output[QBit], angles: QParam[Array[float, 3]]) -> None:
    allocate(1, q)
    U(angles[0], angles[1], angles[2], 0, q)


@cfunc
def cmain() -> None:
    res = vqe(
        HAMILTONIAN,
        False,
        [],
        optimizer=Optimizer.COBYLA,
        max_iteration=1000,
        tolerance=0.001,
        step_size=0,
        skip_compute_variance=False,
        alpha_cvar=1.0,
    )
    save({"result": res})


qmod = create_model(main, classical_execution_function=cmain)
qprog = synthesize(qmod)
show(qprog)
res = execute(qprog)
vqe_result = res.result()[0].value
print(vqe_result.energy, vqe_result.optimal_parameters, vqe_result.eigenstate)